# Import Packages

In [89]:
import pandas as pd
import glob as glob
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

# Import Dataset

In [90]:
df_jams = pd.read_csv("../data/raw/aggregate_median_jams_Kota Bogor.csv")
df_alerts = pd.read_csv("../data/raw/aggregate_alerts_Kota Bogor.csv")
df_irre = pd.read_csv("../data/raw/aggregate_median_irregularities_Kota Bogor.csv")
public_holiday = pd.read_csv("../data/raw/holiday.txt",
                             sep='\t')

df_jams.drop("Unnamed: 0", axis=1, inplace=True)

## Checking data on Alerts dataset & Info

In [91]:
df_jams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102322 entries, 0 to 102321
Data columns (total 13 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   time                       102322 non-null  object 
 1   kemendagri_kabupaten_kode  102322 non-null  float64
 2   kemendagri_kabupaten_nama  102322 non-null  object 
 3   street                     100529 non-null  object 
 4   level                      102322 non-null  int64  
 5   median_length              102322 non-null  float64
 6   median_delay               102322 non-null  float64
 7   median_speed_kmh           102322 non-null  float64
 8   total_records              102322 non-null  int64  
 9   id                         102322 non-null  int64  
 10  date                       102322 non-null  object 
 11  median_level               102322 non-null  float64
 12  geometry                   102322 non-null  object 
dtypes: float64(5), int64(3), obje

In [92]:
df_alerts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9762 entries, 0 to 9761
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         9762 non-null   int64  
 1   time                       9762 non-null   object 
 2   kemendagri_kabupaten_kode  9762 non-null   float64
 3   kemendagri_kabupaten_nama  9762 non-null   object 
 4   street                     8827 non-null   object 
 5   type                       9762 non-null   object 
 6   avg_location               9762 non-null   object 
 7   total_records              9762 non-null   int64  
 8   date                       9762 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 686.5+ KB


In [93]:
df_jams.sample(10)

,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,median_length,median_delay,median_speed_kmh,total_records,id,date,median_level,geometry
26765,2022-07-22 19:00:00.000,32.71,KOTA BOGOR,Orchard Walk,2,558.0,117.5,9.504999,12,34494515,2022-07-22,2.0,"MULTILINESTRING ((106.794838 -6.630271, 106.79..."
15883,2022-07-16 13:00:00.000,32.71,KOTA BOGOR,Surya Kencana,3,1165.0,376.0,7.160000,27,34104706,2022-07-16,3.0,"MULTILINESTRING ((106.799024 -6.603009, 106.79..."
4683,2022-07-09 00:00:00.000,32.71,KOTA BOGOR,Tol Jagorawi,2,1861.0,75.0,29.640000,5,33668185,2022-07-09,2.0,"MULTILINESTRING ((106.838171 -6.629353, 106.83..."
75952,2022-08-20 17:00:00.000,32.71,KOTA BOGOR,Semplak Raya,4,308.0,516.0,1.520000,37,36310487,2022-08-20,4.0,"MULTILINESTRING ((106.761266 -6.553015, 106.76..."
21298,2022-07-19 09:00:00.000,32.71,KOTA BOGOR,Mayjen Ishak Djuarsa,1,1188.0,79.5,22.830002,8,34321964,2022-07-19,1.0,"MULTILINESTRING ((106.777149 -6.5935, 106.7751..."
77870,2022-08-21 21:00:00.000,32.71,KOTA BOGOR,N9 Raden KH Abdullah bin Nuh,3,799.0,90.5,14.690001,30,36372154,2022-08-21,3.0,"MULTILINESTRING ((106.759845 -6.567672, 106.76..."
3483,2022-07-08 11:00:00.000,32.71,KOTA BOGOR,Tol Lingkar Luar Bogor,3,1719.0,164.5,25.130001,6,33598918,2022-07-08,3.0,"MULTILINESTRING ((106.786987 -6.559893, 106.78..."
47145,2022-08-03 12:00:00.000,32.71,KOTA BOGOR,Pahlawan,2,862.0,79.5,16.870000,18,35269963,2022-08-03,2.0,"MULTILINESTRING ((106.794985 -6.607735, 106.79..."
51271,2022-08-05 19:00:00.000,32.71,KOTA BOGOR,Padi,2,307.0,74.0,8.780000,19,35392131,2022-08-05,2.0,"MULTILINESTRING ((106.815874 -6.59899, 106.815..."
73766,2022-08-19 13:00:00.000,32.71,KOTA BOGOR,Subur,3,351.0,125.0,6.470000,10,36242789,2022-08-19,3.0,"MULTILINESTRING ((106.782734 -6.539231, 106.78..."


# Data Quality Checking & Feature Transformation

## Jams Data

In [94]:
df_jams.isna().sum()

time                            0
kemendagri_kabupaten_kode       0
kemendagri_kabupaten_nama       0
street                       1793
level                           0
median_length                   0
median_delay                    0
median_speed_kmh                0
total_records                   0
id                              0
date                            0
median_level                    0
geometry                        0
dtype: int64

In [95]:
df_jams.duplicated().sum()

0

In [96]:
df_jams.dropna(subset=['street'], axis=0, inplace=True)

## Alerts Data

In [97]:
df_alerts.isna().sum()

id                             0
time                           0
kemendagri_kabupaten_kode      0
kemendagri_kabupaten_nama      0
street                       935
type                           0
avg_location                   0
total_records                  0
date                           0
dtype: int64

In [98]:
df_alerts.sample(5)

,id,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,type,avg_location,total_records,date
3896,6051242,2022-07-31 16:00:00.000,32.71,KOTA BOGOR,N8 Jalan Raya Bogor,JAM,"[106.82257299999998, -6.548803000000003]",15,2022-07-31
7710,6160345,2022-08-24 06:00:00.000,32.71,KOTA BOGOR,Semplak Raya,JAM,"[106.76047100000004, -6.543224000000001]",30,2022-08-24
6335,6116585,2022-08-14 15:00:00.000,32.71,KOTA BOGOR,NaN,WEATHERHAZARD,"[106.80146549999995, -6.55521700000001]",120,2022-08-14
2619,6018144,2022-07-24 17:00:00.000,32.71,KOTA BOGOR,N9 KS Tubun,JAM,"[106.80978899999992, -6.566521]",45,2022-07-24
7039,6137822,2022-08-19 17:00:00.000,32.71,KOTA BOGOR,N9 Ir Haji Juanda,JAM,"[106.79715, -6.603894]",4,2022-08-19


In [99]:
# focus on jam only

df_alerts = df_alerts[df_alerts['type']=='JAM'].copy() 

In [100]:
df_alerts.shape

(5400, 9)

## Irregularities in Bogor

In [101]:
df_irre.sample(5)

,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,jam_level,median_length,median_delay_seconds,median_regular_speed,total_records,cause_type,median_seconds,median_speed,date,median_jam_level,id,geometry
1438,2022-07-30 17:00:00.000,32.71,KOTA BOGOR,N9 Jalan Raya Dramaga,3,1242.0,427.0,14.53,3,NaN,553.0,8.08,2022-07-30,3.0,1533820,"MULTILINESTRING ((106.751825 -6.571448, 106.75..."
2396,2022-08-12 17:00:00.000,32.71,KOTA BOGOR,N9 Jalan Raya Tajur,4,1608.0,609.0,25.15,1,NaN,738.0,7.84,2022-08-12,4.0,1552293,"LINESTRING (106.835719 -6.640913, 106.836749 -..."
1230,2022-07-27 18:00:00.000,32.71,KOTA BOGOR,Jalan Empang,4,592.0,534.0,7.28,6,NaN,624.0,3.41,2022-07-27,4.0,1528228,"MULTILINESTRING ((106.796754 -6.604469, 106.79..."
1304,2022-07-29 19:00:00.000,32.71,KOTA BOGOR,Kedunghalang Raya,2,1066.0,149.5,22.93,8,NaN,270.0,14.19,2022-07-29,2.0,1530811,"MULTILINESTRING ((106.807804 -6.553153, 106.80..."
3232,2022-08-27 15:00:00.000,32.71,KOTA BOGOR,Kapten Yusuf,4,765.0,535.0,11.77,3,NaN,632.0,4.35,2022-08-27,4.0,1569759,"MULTILINESTRING ((106.787318 -6.614671, 106.78..."


## Holiday data

In [106]:
public_holiday

,Date,Weekday,Note,Holiday
0,"Jan 01, 2022",Saturday,Public Holiday,New Year's Day
1,"Feb 01, 2022",Tuesday,Public Holiday,Chinese New Year
2,"Feb 28, 2022",Monday,Public Holiday,Isra and Mi'raj
3,"Mar 03, 2022",Thursday,Public Holiday,Nyepi
4,"Apr 15, 2022",Friday,Public Holiday,Good Friday
5,"May 01, 2022",Sunday,Public Holiday,Labor Day
6,"May 02, 2022",Monday,Public Holiday,Eid al-Fitr
7,"May 03, 2022",Tuesday,Public Holiday,Eid al-Fitr Holiday
8,"May 16, 2022",Monday,Public Holiday,Buddha Day
9,"May 26, 2022",Thursday,Public Holiday,Ascension Day


In [107]:
public_holiday.rename(columns={"Date":"time"}, inplace=True)
public_holiday['time'] = pd.to_datetime(public_holiday["time"], format='%b %d, %Y') #reformating date

In [111]:
public_holiday.sample(5)

,time,Weekday,Note,Holiday
0,2022-01-01,Saturday,Public Holiday,New Year's Day
9,2022-05-26,Thursday,Public Holiday,Ascension Day
14,2022-10-08,Saturday,Public Holiday,Prophet's Birthday
13,2022-08-17,Wednesday,Public Holiday,Independence Day
4,2022-04-15,Friday,Public Holiday,Good Friday


# Transforming Data

## Transforming Timeseries Data

In [103]:
df_jams['time'] = pd.to_datetime(df_jams['time'], format="%Y-%m-%d %H:%M:%S.%f")

In [123]:
df_jams['day_name'] = df_jams['time'].dt.day_name()
df_jams['hour'] = df_jams['time'].dt.hour
df_jams['weekday'] = df_jams['time'].dt.weekday
df_jams['isweekend'] = np.where(((df_jams['weekday']==5) | (df_jams['weekday']==6)),
                                1,0)
df_jams['month'] = df_jams['time'].dt.month
df_jams['pub_holiday'] = np.where((df_jams['time'].dt.date.astype(str)).isin(public_holiday['time'].astype(str)),
                                  1,0)

df_jams.sample(5)

,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,median_length,median_delay,median_speed_kmh,total_records,id,date,median_level,geometry,day_name,hour,weekday,isweekend,month,pub_holiday
42290,2022-07-31 14:00:00,32.71,KOTA BOGOR,Merdeka,4,215.0,160.0,4.160,17,35096209,2022-07-31,4.0,"MULTILINESTRING ((106.787715 -6.584949, 106.78...",Sunday,14,6,1,7,0
96020,2022-09-01 18:00:00,32.71,KOTA BOGOR,Kedunghalang Raya,2,1587.0,217.0,14.775,10,37013175,2022-09-01,2.0,"MULTILINESTRING ((106.810567 -6.555411, 106.80...",Thursday,18,3,0,9,0
95723,2022-09-01 16:00:00,32.71,KOTA BOGOR,Cifor,4,642.0,418.0,4.560,5,37012877,2022-09-01,4.0,"MULTILINESTRING ((106.750796 -6.563663, 106.75...",Thursday,16,3,0,9,0
53950,2022-08-07 08:00:00,32.71,KOTA BOGOR,N9 Jalan Raya Tajur,1,2097.0,84.0,28.440,25,35544856,2022-08-07,1.0,"MULTILINESTRING ((106.824909 -6.63156, 106.824...",Sunday,8,6,1,8,0
25130,2022-07-21 17:00:00,32.71,KOTA BOGOR,N9 Jalan Raya Pajajaran,4,775.0,353.0,6.640,25,34436648,2022-07-21,4.0,"MULTILINESTRING ((106.807526 -6.576516, 106.80...",Thursday,17,3,0,7,0


# Generate Interim Data

In [130]:
df_jams.to_parquet('../data/interim/interim_data.parquet', index=False)